## 读取MovieLens 数据

In [6]:
import pandas
training_set = pandas.read_csv('../ml-100k/u1.base', sep = '\t', names = ['userid', 'itemid', 'rating', 'tm'])
training_set.head()

,userid,itemid,rating,tm
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [7]:
import pandas
test_set = pandas.read_csv('../ml-100k/u1.test', sep = '\t', names = ['userid', 'itemid', 'rating', 'tm'])
test_set.head()

,userid,itemid,rating,tm
0,1,6,5,887431973
1,1,10,3,875693118
2,1,12,5,878542960
3,1,14,5,874965706
4,1,17,3,875073198


## 计算电影与使用者数量

In [14]:
n_movies = max(max(training_set.itemid.tolist()), max(test_set.itemid.tolist()))
n_movies

1682

In [15]:
n_users= max(max(training_set.userid.tolist()), max(test_set.userid.tolist()))
n_users

943

## 建立训练数据集矩阵

In [59]:
import numpy as np
training_m = np.zeros((n_users, n_movies))
for rec in training_set.iterrows():
    training_m[rec[1].userid - 1 , rec[1].itemid - 1] = 1

In [60]:
training_m.shape

(943, 1682)

## 建立测试数据集矩阵

In [61]:
import numpy as np
test_m = np.zeros((n_users, n_movies))
for rec in test_set.iterrows():
    test_m[rec[1].userid - 1 , rec[1].itemid - 1] = 1

In [63]:
test_m.shape

(943, 1682)

## 建立Autoencoders

In [65]:
from keras.layers import Input, Dense
from keras.models import Model

encoding_dim = 50  
input_data = Input(shape=(n_users,))
encoded = Dense(encoding_dim, activation='softmax')(input_data)
decoded = Dense(n_users)(encoded)

autoencoder = Model(input_data, decoded)
autoencoder.compile(optimizer='adam', loss='mean_absolute_error')

## 训练Autoencoders

In [66]:
autoencoder.fit(m.T, m.T,
                epochs=200,
                batch_size=32,
                shuffle=True,
                validation_data=(test_m.T, test_m.T) )

Train on 1682 samples, validate on 1682 samples
Epoch 1/200
1682/1682 [==============================] - 0s 228us/step - loss: 0.0518 - val_loss: 0.0132
Epoch 2/200
1682/1682 [==============================] - 0s 120us/step - loss: 0.0511 - val_loss: 0.0131
Epoch 3/200
1682/1682 [==============================] - 0s 118us/step - loss: 0.0510 - val_loss: 0.0130
Epoch 4/200
1682/1682 [==============================] - 0s 119us/step - loss: 0.0509 - val_loss: 0.0130
Epoch 5/200
1682/1682 [==============================] - 0s 118us/step - loss: 0.0508 - val_loss: 0.0129
Epoch 6/200
1682/1682 [==============================] - 0s 121us/step - loss: 0.0507 - val_loss: 0.0129
Epoch 7/200
1682/1682 [==============================] - 0s 119us/step - loss: 0.0507 - val_loss: 0.0129
Epoch 8/200
1682/1682 [==============================] - 0s 118us/step - loss: 0.0507 - val_loss: 0.0129
Epoch 9/200
1682/1682 [==============================] - 0s 120us/step - loss: 0.0506 - val_loss: 0.0129
Epoch 1

1682/1682 [==============================] - 0s 127us/step - loss: 0.0481 - val_loss: 0.0131
Epoch 78/200
1682/1682 [==============================] - 0s 130us/step - loss: 0.0480 - val_loss: 0.0131
Epoch 79/200
1682/1682 [==============================] - 0s 119us/step - loss: 0.0480 - val_loss: 0.0131
Epoch 80/200
1682/1682 [==============================] - 0s 117us/step - loss: 0.0480 - val_loss: 0.0131
Epoch 81/200
1682/1682 [==============================] - 0s 123us/step - loss: 0.0480 - val_loss: 0.0132
Epoch 82/200
1682/1682 [==============================] - 0s 116us/step - loss: 0.0480 - val_loss: 0.0131
Epoch 83/200
1682/1682 [==============================] - 0s 117us/step - loss: 0.0480 - val_loss: 0.0132
Epoch 84/200
1682/1682 [==============================] - 0s 119us/step - loss: 0.0480 - val_loss: 0.0132
Epoch 85/200
1682/1682 [==============================] - 0s 124us/step - loss: 0.0480 - val_loss: 0.0132
Epoch 86/200
1682/1682 [==============================] - 0

Epoch 154/200
1682/1682 [==============================] - 0s 125us/step - loss: 0.0477 - val_loss: 0.0133
Epoch 155/200
1682/1682 [==============================] - 0s 124us/step - loss: 0.0477 - val_loss: 0.0133
Epoch 156/200
1682/1682 [==============================] - 0s 124us/step - loss: 0.0477 - val_loss: 0.0133
Epoch 157/200
1682/1682 [==============================] - 0s 126us/step - loss: 0.0477 - val_loss: 0.0133
Epoch 158/200
1682/1682 [==============================] - 0s 124us/step - loss: 0.0477 - val_loss: 0.0134
Epoch 159/200
1682/1682 [==============================] - 0s 129us/step - loss: 0.0477 - val_loss: 0.0133
Epoch 160/200
1682/1682 [==============================] - 0s 125us/step - loss: 0.0477 - val_loss: 0.0133
Epoch 161/200
1682/1682 [==============================] - 0s 130us/step - loss: 0.0477 - val_loss: 0.0134
Epoch 162/200
1682/1682 [==============================] - 0s 124us/step - loss: 0.0477 - val_loss: 0.0134
Epoch 163/200
1682/1682 [============

## 建立 Encoder

In [68]:
encoder = Model(input_data, encoded)

In [72]:
encoded_data = encoder.predict(training_m.T)

In [73]:
encoded_data.shape

(1682, 50)

## 建立 Decoder

In [74]:
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [77]:
pred = decoder.predict(encoded_data)

## 检视原始与重建数据

In [94]:
sum((pred[:,0] > 0.001).astype(int) == training_m.T[:,0]) / len(training_m.T[:,0])

0.920332936979786

In [95]:
((pred > 0.001).astype(int) == training_m.T).sum() / (1682 * 943)

0.9473389882014418

In [96]:
pred[ training_m.T == 0] = 0
mse = np.mean(np.power(training_m.T.flatten() - pred.flatten(), 2))
mse

0.04412707051763893

## 套用到测试数据集

In [81]:
encoded_test_data = encoder.predict(test_m.T)

In [82]:
pred_test = decoder.predict(encoded_test_data)

In [83]:
((pred_test > 0.001).astype(int) == test_m.T).sum() / (1682 * 943)

0.963634667107153

In [93]:
pred_test[ test_m.T == 0] = 0
mse = np.mean(np.power(test_m.T.flatten() - pred_test.flatten(), 2))
mse

0.012577480334775201